# CalculoRubroBase

In [105]:
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime
import import_ipynb
from shared_functions1 import DataframeLoader, Shared
from calculosba_areacalculo import AreaCalculoSBA

# import import_ipynb
base_dir = 'C:\\Users\\Crisp\\Documents\\QualitySoftGroup\\Proyectos\\Liquidations-Incentives-Jupyter'
sub_direct = 'archivos-fuente-95(1)'
folder_bbraun_source = os.path.join(base_dir, 'source', sub_direct)
folder_bbraun_output = os.path.join(base_dir, 'output')
print(folder_bbraun_source)

C:\Users\Crisp\Documents\QualitySoftGroup\Proyectos\Liquidations-Incentives-Jupyter\source\archivos-fuente-95(1)


In [106]:
dataframe_loader = DataframeLoader(
    base_dir=base_dir,
    sub_direct=sub_direct
)
shared = Shared()
dataframe_loader.cargar_dataframes(base_dir, folder_bbraun_source)

(        Contexto      Fecha                             Variable  \
 0     1013583607 2023-01-01                ObjetivosCualitativos   
 1          gu_10 2023-01-01          VentaFacturadaUnidadNegocio   
 2        gu_10_1 2023-01-01         VentaCobradaUnidadNegocioSub   
 3        gu_10_2 2023-01-01         VentaCobradaUnidadNegocioSub   
 4          BBMCO 2023-01-01              VentaFacturadaBraunPais   
 ...          ...        ...                                  ...   
 8880    94487610 2023-11-01  VentaFacturadaPortafolioZonaCluster   
 8881          10 2023-11-01             VentaCobradaBraunCluster   
 8882     9990010 2023-11-01               VentaFacturadaZonaPais   
 8883          11 2023-11-01             VentaCobradaBraunCluster   
 8884     9990011 2023-11-01               VentaFacturadaZonaPais   
 
       PorcentajeCumplimiento          Real   Presupuesto CodigoEmpleado  \
 0                       0.94  0.000000e+00  0.000000e+00     1013583607   
 1                

In [130]:
class CalculoRubroBase:
    
    dataframe_loader = None
    grupo_empleados = None
    nombre_rubro = None
    df_real = None
    df_presupuesto = None
    tipo_venta = None
    columnas_extra = None # sobreescribir_resultados_cualitativos
    columnas_extra_merge = None  # sobreescribir_resultados_cualitativos
    columnas_source = None  # Columnas para merge entre SBA y Real o Presupuesto
    area_calculo_sba_completo = None
    main_real = None
    main_presupuesto = None
    
    def __init__(self, dataframe_loader):
        self.dataframe_loader = dataframe_loader

    def get_grupo_empleados_by_nombre_rubro(self, nombre_rubro=None):
        if nombre_rubro:
            self.nombre_rubro = nombre_rubro
        self.grupo_empleados = self.dataframe_loader.df_parrillas[self.dataframe_loader.df_parrillas['Variable'] == self.nombre_rubro]
        return self.grupo_empleados

#     def get_grupos_empleados(self):
#         grupo_empleados = list()
#         for nombre_rubro in self.nombres_rubros:
#             grupo_empleados.append(self.get_grupo_empleados_by_nombre_rubro(nombre_rubro))
#         self.grupo_empleados = pd.concat(grupo_empleados)
    
    def get_tipos_empleados(self):
        tipos_empleados = self.grupo_empleados.merge(
            self.dataframe_loader.df_empleados,
            how = 'left'
        )['TipoEmpleado'].dropna().drop_duplicates()
        
        print(tipos_empleados)
        
        return tipos_empleados
        
    def calcular_centro_costo_by_tipo_empleado(self, main, tipo_empleado = None, *args, **kwargs):
        return dataframe_loader.f_calculo_centro_costo_variable_division_sub(main, self.nombre_rubro, tipo_empleado)
    
    def calcular_centro_costo(self, main):
        main_list = list()
        for tipo_empleado in self.get_tipos_empleados():
             main_list.append(calcular_centro_costo_by_tipo_empleado(main, tipo_empleado))
        return pd.concat(main_list)
    
    def calcular(self, grupo_datos, *args, **kwargs):
        raise NotImplementedError("Debe implementarse un método de cálculo específico.")
        
    def generacion_columnas(self, main):
        return main[['Contexto', 'Variable', 'Fecha', 'Real', 'Presupuesto', 'PorcentajeCumplimiento']]

    def calculo_variable(self, main):
        main['Real'] = main['Real'].transform(pd.Series.cumsum)
        main['Presupuesto'] = main['Presupuesto'].transform(pd.Series.cumsum)
        print("self.columnas_extra_merge:", self.columnas_extra_merge)
        if self.columnas_extra_merge:
            print("Entró")
            main['Real'] = main.groupby(self.columnas_extra_merge)['Real'].transform(pd.Series.cumsum)
            main['Presupuesto'] = main.groupby(self.columnas_extra_merge)['Presupuesto'].transform(pd.Series.cumsum)
        main['PorcentajeCumplimiento'] = main['Real'] / main['Presupuesto']
        main['PorcentajeCumplimiento'] = main['PorcentajeCumplimiento'].apply(lambda x: round(x, 2))
        main['Variable'] = self.nombre_rubro        
        return main    

    def get_main(self):
        main_real = self.df_real.groupby(self.group_by, as_index=False).agg({self.tipo_venta: 'sum'})
        main_real['Real'] = main_real[self.tipo_venta]
        main_real.drop(columns=[self.tipo_venta], inplace=True)
        self.main_real = main_real
        
        main_presupuesto = self.df_presupuesto.groupby(self.group_by, as_index=False).agg({self.tipo_venta: 'sum'})
        main_presupuesto['Presupuesto'] = main_presupuesto[self.tipo_venta]
        main_presupuesto.drop(columns=[self.tipo_venta], inplace=True)
        self.main_presupuesto = main_presupuesto

        main = pd.merge(
            left=main_presupuesto,
            right=main_real,
            on=self.group_by,
            how='left'
        )
        return main

    def get_area_calculo_sba_completo(self, grupo_empleados, c_area_calculo_sba):
        self.area_calculo_sba_completo = pd.DataFrame(columns=self.columnas_source_merge)
        if grupo_empleados is None:
#             grupo_empleados = self.get_grupos_empleados()
            grupo_empleados = self.get_grupo_empleados_by_nombre_rubro()
        
        tipos_empleados = self.get_tipos_empleados()
        
        area_calculo_sba_completo_list = list()
        
        for tipo_empleado in tipos_empleados:
            area_calculo_sba_completo_list.append(c_area_calculo_sba.get_area_calculo_sba_completo_by_tipo_empleado(tipo_empleado))
        if area_calculo_sba_completo_list:
            self.area_calculo_sba_completo = pd.concat(area_calculo_sba_completo_list)
    
class VentaFacturadaZonaPais(CalculoRubroBase):
    def calcular(self, grupo_datos, *args, **kwargs):
        # Implementa el cálculo específico aquí
        pass

class VentaFacturadaPortafolioCluster(CalculoRubroBase):
    def calcular(self, grupo_datos, *args, **kwargs):
        # Implementa el cálculo específico aquí
        pass

In [131]:
class ObjetivosCualitativos(CalculoRubroBase):
    
    nombre_rubro = 'ObjetivosCualitativos'
    df_objetivos_cualitativos = None
    df_empleados = None
    df_venta_recaudo_real = None

    def calcular(self, grupo_empleados = None, *args, **kwargs):
        if grupo_empleados is None:
             grupo_empleados = self.get_grupo_empleados_by_nombre_rubro()
        self.df_objetivos_cualitativos = self.dataframe_loader.df_objetivos_cualitativos
        self.df_empleados = grupo_empleados
        self.df_venta_recaudo_real = self.dataframe_loader.df_venta_recaudo_real

        cumplimiento_objetivos_cualitativos = pd.DataFrame()
        cumplimiento_objetivos_cualitativos['CodigoEmpleado'] = self.df_empleados['CodigoEmpleado']
        cumplimiento_objetivos_cualitativos['merge'] = 1
        ##
        fechas = pd.DataFrame(self.df_venta_recaudo_real['Fecha'].drop_duplicates())
        fechas['merge'] = 1
        ##
        cumplimiento_objetivos_cualitativos = cumplimiento_objetivos_cualitativos.merge(
            fechas,
            on='merge',
            how='left'
        )
        cumplimiento_objetivos_cualitativos.drop(columns='merge', inplace=True)

        cumplimiento_objetivos_cualitativos = pd.merge(
            left=cumplimiento_objetivos_cualitativos,
            right=self.df_objetivos_cualitativos,
            on=['CodigoEmpleado', 'Fecha'],
            how='left'
        )

        cumplimiento_objetivos_cualitativos['Variable'] = self.nombre_rubro
        cumplimiento_objetivos_cualitativos['PorcentajeCumplimiento'].fillna(0.94, inplace=True)
        cumplimiento_objetivos_cualitativos.rename(columns={'CodigoEmpleado': 'Contexto'}, inplace=True)
        return cumplimiento_objetivos_cualitativos

In [132]:
class VentaCobradaZonaBraunCrossDivisional(CalculoRubroBase):
    
    nombre_rubro = 'VentaCobradaZonaBraunCrossDivisional'
    columnas_extra_merge = 'Zona' 
    
    def calcular(self, grupo_empleados = None, *args, **kwargs):       
        cumplimiento_recaudo_por_zona = self.calculo_variable(self.dataframe_loader.df_recaudo_por_zona.copy())       
        cumplimiento_recaudo_por_zona.rename(columns={'Zona': 'Contexto'}, inplace=True)
        return self.generacion_columnas(cumplimiento_recaudo_por_zona)
        

In [133]:
class VentaCobradaBraunPais(CalculoRubroBase):
    
    nombre_rubro = 'VentaCobradaBraunPais'
    tipo_venta = 'Recaudo' #Cobrada
    group_by=['Fecha'] 
    
    def calcular(self, grupo_empleados = None, *args, **kwargs):
        # recaudo real pais
        self.df_real = self.dataframe_loader.df_venta_recaudo_real
        self.df_presupuesto = self.dataframe_loader.df_venta_recaudo_presupuesto
        cumplimiento_recaudo_nivel_pais = self.get_main()
        cumplimiento_recaudo_nivel_pais['Real'].fillna(0.0, inplace=True)
        cumplimiento_recaudo_nivel_pais = self.calculo_variable(cumplimiento_recaudo_nivel_pais)
        cumplimiento_recaudo_nivel_pais['Contexto'] = 'Pais'
        return self.generacion_columnas(cumplimiento_recaudo_nivel_pais)


In [134]:
venta_cobrada_braun_pais = VentaCobradaBraunPais(dataframe_loader)
venta_cobrada_braun_pais.calcular()

self.columnas_extra_merge: None


,Contexto,Variable,Fecha,Real,Presupuesto,PorcentajeCumplimiento
0,Pais,VentaCobradaBraunPais,2023-01-01,1.356524e+10,1.496945e+10,0.91
1,Pais,VentaCobradaBraunPais,2023-02-01,2.665112e+10,3.087311e+10,0.86
2,Pais,VentaCobradaBraunPais,2023-03-01,3.992673e+10,4.648396e+10,0.86
3,Pais,VentaCobradaBraunPais,2023-04-01,5.525573e+10,6.067049e+10,0.91
4,Pais,VentaCobradaBraunPais,2023-05-01,7.298270e+10,7.454188e+10,0.98
5,Pais,VentaCobradaBraunPais,2023-06-01,8.934016e+10,8.932282e+10,1.00
6,Pais,VentaCobradaBraunPais,2023-07-01,1.033452e+11,1.024664e+11,1.01
7,Pais,VentaCobradaBraunPais,2023-08-01,1.168169e+11,1.178687e+11,0.99
8,Pais,VentaCobradaBraunPais,2023-09-01,1.348872e+11,1.325749e+11,1.02
9,Pais,VentaCobradaBraunPais,2023-10-01,1.512564e+11,1.481929e+11,1.02


In [135]:
venta_cobrada_braun_pais = VentaCobradaBraunPais(dataframe_loader)
venta_cobrada_braun_pais.calcular()

self.columnas_extra_merge: None


,Contexto,Variable,Fecha,Real,Presupuesto,PorcentajeCumplimiento
0,Pais,VentaCobradaBraunPais,2023-01-01,1.356524e+10,1.496945e+10,0.91
1,Pais,VentaCobradaBraunPais,2023-02-01,2.665112e+10,3.087311e+10,0.86
2,Pais,VentaCobradaBraunPais,2023-03-01,3.992673e+10,4.648396e+10,0.86
3,Pais,VentaCobradaBraunPais,2023-04-01,5.525573e+10,6.067049e+10,0.91
4,Pais,VentaCobradaBraunPais,2023-05-01,7.298270e+10,7.454188e+10,0.98
5,Pais,VentaCobradaBraunPais,2023-06-01,8.934016e+10,8.932282e+10,1.00
6,Pais,VentaCobradaBraunPais,2023-07-01,1.033452e+11,1.024664e+11,1.01
7,Pais,VentaCobradaBraunPais,2023-08-01,1.168169e+11,1.178687e+11,0.99
8,Pais,VentaCobradaBraunPais,2023-09-01,1.348872e+11,1.325749e+11,1.02
9,Pais,VentaCobradaBraunPais,2023-10-01,1.512564e+11,1.481929e+11,1.02


In [136]:
class VentaUnidadNegocio(CalculoRubroBase):
    
#     venta_recaudo_presupuesto_original_gu_real = None
#     venta_recaudo_presupuesto_original_gu_presupuesto = None
#     columnas_extra = None
#     columnas_extra_merge = None
    columnas_source = ['Division', 'GrupoProducto', 'SBA', 'Fecha', 'Venta', 'Recaudo']
    columnas_source_merge = ['Division', 'GrupoProducto', 'SBA']
    
    nombres_rubros = ['VentaFacturadaUnidadNegocio', 'VentaFacturadaUnidadNegocioSub', 'VentaCobradaUnidadNegocio', 'VentaCobradaUnidadNegocioSub']
        
    def get_main(self):
        main = super().get_main()
        main['Real'].fillna(0.0, inplace=True)
        return main
    
    def sobreescribir_resultados_cualitativos(self, main, tipo_empleado = None):
        ### Cálculo de resultados cualitativos precargados
        return self.dataframe_loader.sobreescribir_resultados_cualitativos(
            main=main,
            variable=self.nombre_rubro,
            columnas_extra=self.columnas_extra, 
            columnas_extra_merge=self.columnas_extra_merge,
            pre_merge_lambda=lambda x: x[x['TipoEmpleado'] == 'GU']
        )
        
    def calculo_variable(self, main):        
        main = super().calculo_variable(main)
        return self.sobreescribir_resultados_cualitativos(main)
    
    def calcular(self, grupo_empleados = None, *args, **kwargs):
        self.get_area_calculo_sba_completo(grupo_empleados, args[0])
#         area_calculo_sba_completo_gu = area_calculo_sba_completo[area_calculo_sba_completo['TipoEmpleado']==tipo_empleado]
        
        print(self.dataframe_loader.df_venta_recaudo_real.info())
        
        #Se halla la informacion de la venta y recaudo real de los representantes de ventas de los GU
        self.df_real = self.area_calculo_sba_completo.merge(
            self.dataframe_loader.df_venta_recaudo_real[self.columnas_source],
            on=self.columnas_source_merge
        )

        #Se halla la informacion de la venta y recaudo presupuesto de los representantes de ventas de los GU
        self.df_presupuesto = self.area_calculo_sba_completo.merge(
            self.dataframe_loader.df_venta_recaudo_presupuesto[self.columnas_source],
            on=self.columnas_source_merge
        )
        
        return self.df_real, self.df_presupuesto

In [137]:
venta_unidad_negocio = VentaUnidadNegocio(dataframe_loader)

df_area_calculo_sba = dataframe_loader.df_area_calculo_sba
c_area_calculo_sba = AreaCalculoSBA(dataframe_loader)
area_calculo_sba_completo = c_area_calculo_sba.get_area_calculo_sba_completo()

In [138]:
real, presupuesto = venta_unidad_negocio.calcular(None, c_area_calculo_sba)

Series([], Name: TipoEmpleado, dtype: object)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10706 entries, 0 to 10705
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   CodigoEmpleado  10706 non-null  object        
 1   Division        10706 non-null  object        
 2   Cluster         10706 non-null  object        
 3   ZonaGeografica  10706 non-null  object        
 4   GrupoProducto   10706 non-null  object        
 5   Canal           10706 non-null  object        
 6   SBA             10706 non-null  object        
 7   Fecha           10706 non-null  datetime64[ns]
 8   Moneda          10706 non-null  object        
 9   Venta           10706 non-null  float64       
 10  Recaudo         10706 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(8)
memory usage: 920.2+ KB
None


In [139]:
real

,Division,GrupoProducto,SBA,Fecha,Venta,Recaudo


In [140]:
presupuesto

,Division,GrupoProducto,SBA,Fecha,Venta,Recaudo


In [141]:
real

,Division,GrupoProducto,SBA,Fecha,Venta,Recaudo


In [142]:
presupuesto

,Division,GrupoProducto,SBA,Fecha,Venta,Recaudo


In [143]:
class VentaFacturadaUnidadNegocio(VentaUnidadNegocio):
    
    nombre_rubro = 'VentaFacturadaUnidadNegocio'
    tipo_venta = 'Venta'
    cumplimiento_venta_por_unidad_negocio_gu = None
    group_by = ['AreaCalculo', 'Fecha']
    columnas_extra = ['TipoEmpleado', 'AreaCalculo']
    columnas_extra_merge = ['AreaCalculo']
    
    def get_contexto(self, cumplimiento_venta_por_unidad_negocio_gu):
        return 'gu_' + cumplimiento_venta_por_unidad_negocio_gu['AreaCalculo']
        
    def calcular(self, grupo_empleados = None, *args, **kwargs):
        c_area_calculo_sba = args[0]
        super().calcular(None, c_area_calculo_sba)
        cumplimiento_venta_por_unidad_negocio_gu = self.calculo_variable(self.get_main())

        ### Cálculo de resultados cualitativos precargados
        cumplimiento_venta_por_unidad_negocio_gu['AreaCalculo'] = cumplimiento_venta_por_unidad_negocio_gu['AreaCalculo'].astype('str')
        
        cumplimiento_venta_por_unidad_negocio_gu['Contexto'] = self.get_contexto(cumplimiento_venta_por_unidad_negocio_gu)
        
        # Checkpoint Calculo
        return self.generacion_columnas(cumplimiento_venta_por_unidad_negocio_gu)

In [144]:
venta_facturada_unidad_negocio = VentaFacturadaUnidadNegocio(dataframe_loader)
venta_facturada_unidad_negocio.calcular(None, c_area_calculo_sba)

0    GU
Name: TipoEmpleado, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10706 entries, 0 to 10705
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   CodigoEmpleado  10706 non-null  object        
 1   Division        10706 non-null  object        
 2   Cluster         10706 non-null  object        
 3   ZonaGeografica  10706 non-null  object        
 4   GrupoProducto   10706 non-null  object        
 5   Canal           10706 non-null  object        
 6   SBA             10706 non-null  object        
 7   Fecha           10706 non-null  datetime64[ns]
 8   Moneda          10706 non-null  object        
 9   Venta           10706 non-null  float64       
 10  Recaudo         10706 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(8)
memory usage: 920.2+ KB
None
self.columnas_extra_merge: ['AreaCalculo']
Entró
['TipoEmpleado', 'AreaCalculo'] ['AreaCalculo'

,Contexto,Variable,Fecha,Real,Presupuesto,PorcentajeCumplimiento
0,gu_1,VentaFacturadaUnidadNegocio,2023-01-01,6.098946e+08,5.390604e+08,1.13
1,gu_1,VentaFacturadaUnidadNegocio,2023-02-01,2.265443e+09,1.641055e+09,1.38
2,gu_1,VentaFacturadaUnidadNegocio,2023-03-01,5.558293e+09,3.724501e+09,1.49
3,gu_1,VentaFacturadaUnidadNegocio,2023-04-01,1.012908e+10,6.503516e+09,1.56
4,gu_1,VentaFacturadaUnidadNegocio,2023-05-01,1.615783e+10,1.016343e+10,1.59
...,...,...,...,...,...,...
103,gu_10,VentaFacturadaUnidadNegocio,2023-08-01,1.511933e+12,1.438558e+12,1.05
104,gu_10,VentaFacturadaUnidadNegocio,2023-09-01,1.702786e+12,1.620288e+12,1.05
105,gu_10,VentaFacturadaUnidadNegocio,2023-10-01,1.894150e+12,1.802511e+12,1.05
106,gu_10,VentaFacturadaUnidadNegocio,2023-11-01,2.086289e+12,1.985220e+12,1.05


In [145]:
# venta_facturada_unidad_negocio = VentaFacturadaUnidadNegocio(dataframe_loader)
# venta_facturada_unidad_negocio.calcular(None, c_area_calculo_sba)

In [146]:
class VentaFacturadaUnidadNegocioSub(VentaUnidadNegocio):
    
    nombre_rubro = 'VentaFacturadaUnidadNegocioSub'
    tipo_venta = 'Venta'
    cumplimiento_venta_por_unidad_negocio_gu_sub = None
    group_by = ['AreaCalculo', 'Consecutivo', 'Fecha', 'Division', 'GrupoProducto']  
    columnas_extra = ['TipoEmpleado', 'AreaCalculo', 'Consecutivo']
    columnas_extra_merge = ['AreaCalculo', 'Consecutivo']  
    
    #MK
        
    def get_contexto(self, cumplimiento_venta_por_unidad_negocio_gu_sub):
        return 'gu_' + cumplimiento_venta_por_unidad_negocio_gu_sub['AreaCalculo'].str.cat(cumplimiento_venta_por_unidad_negocio_gu_sub['Consecutivo'],sep="_")
        
    def calcular(self, grupo_empleados = None, *args, **kwargs):
        c_area_calculo_sba = args[0]
        super().calcular(None, c_area_calculo_sba)

        self.cumplimiento_venta_por_unidad_negocio_gu_sub = self.get_main()
        # Checkpoint Calculo
        
        cumplimiento_venta_por_unidad_negocio_gu_sub = self.calculo_variable(self.get_main())
        
        cumplimiento_venta_por_unidad_negocio_gu_sub['Consecutivo'] = cumplimiento_venta_por_unidad_negocio_gu_sub['Consecutivo'].astype('str')
        cumplimiento_venta_por_unidad_negocio_gu_sub['AreaCalculo'] = cumplimiento_venta_por_unidad_negocio_gu_sub['AreaCalculo'].astype('str')
        
        cumplimiento_venta_por_unidad_negocio_gu_sub['Contexto'] = self.get_contexto(cumplimiento_venta_por_unidad_negocio_gu_sub)
        
        return self.generacion_columnas(cumplimiento_venta_por_unidad_negocio_gu_sub)

    def calcular_centro_costo(self, tipo_empleado = None, *args, **kwargs):
        return super().calcular_centro_costo(cumplimiento_venta_por_unidad_negocio_gu_sub)

In [159]:
class VentaDivision(CalculoRubroBase):
    
    nombres_rubros = ['VentaCobradaDivisionSub', 'VentaCobradaDivision', 'VentaFacturadaDivision', 'VentaFacturadaDivisionSub']
    columnas_source = ['Division', 'GrupoProducto', 'SBA', 'Fecha', 'Venta', 'Recaudo']
    columnas_source_merge = ['Division', 'GrupoProducto', 'SBA']
    
    def get_main(self):
        main = super().get_main()
        main['Real'].fillna(0.0, inplace=True)
        return main      

    def calcular(self, grupo_empleados = None, *args, **kwargs):
        self.get_area_calculo_sba_completo(grupo_empleados, args[0])
#         area_calculo_sba_completo_gu = area_calculo_sba_completo[area_calculo_sba_completo['TipoEmpleado']==tipo_empleado]
        
#         print(self.dataframe_loader.df_venta_recaudo_real.info())
        
        #Se halla la informacion de la venta y recaudo real de los representantes de ventas de los GU
        self.df_real = self.area_calculo_sba_completo.merge(
            self.dataframe_loader.df_venta_recaudo_real[self.columnas_source],
            on=self.columnas_source_merge
        )

        #Se halla la informacion de la venta y recaudo presupuesto de los representantes de ventas de los GU
        self.df_presupuesto = self.area_calculo_sba_completo.merge(
            self.dataframe_loader.df_venta_recaudo_presupuesto[self.columnas_source],
            on=self.columnas_source_merge
        )
        
        return self.df_real, self.df_presupuesto    
    
#     def calcular_centro_costo(self, tipo_empleado = None, *args, **kwargs):
#         return super().calcular_centro_costo(self.cumplimiento_recaudo_por_division_sub_mk)

In [169]:
class VentaCobradaDivision(VentaDivision):

    nombre_rubro = 'VentaCobradaDivision'
    tipo_venta = 'Recaudo'
    cumplimiento_recaudo_por_division = None
    group_by = ['Division', 'Fecha']
    columnas_extra = ['TipoEmpleado', 'AreaCalculo']
    columnas_extra_merge = ['Division']
    
    def get_contexto(self, cumplimiento_venta_por_unidad_negocio_gu):
        return 'gu_' + cumplimiento_venta_por_unidad_negocio_gu['AreaCalculo']
        
    def calcular(self, grupo_empleados = None, *args, **kwargs):
        c_area_calculo_sba = args[0]
        super().calcular(None, c_area_calculo_sba)
        
        self.df_real = self.dataframe_loader.df_venta_recaudo_real[['Division', 'Fecha', 'Recaudo']]
        self.df_presupuesto = self.dataframe_loader.df_venta_recaudo_presupuesto[['Division', 'Fecha', 'Recaudo']]
        
#         cumplimiento_recaudo_por_division = self.calculo_variable(self.get_main())

        cumplimiento_recaudo_por_division = self.get_main()
    
        cumplimiento_recaudo_por_division['Real'] = cumplimiento_recaudo_por_division.groupby(['Division'])['Real'].transform(pd.Series.cumsum)
        cumplimiento_recaudo_por_division['Presupuesto'] = cumplimiento_recaudo_por_division.groupby(['Division'])['Presupuesto'].transform(pd.Series.cumsum)
        cumplimiento_recaudo_por_division['PorcentajeCumplimiento'] = cumplimiento_recaudo_por_division['Real'] / cumplimiento_recaudo_por_division['Presupuesto']
        cumplimiento_recaudo_por_division['PorcentajeCumplimiento'] = cumplimiento_recaudo_por_division['PorcentajeCumplimiento'].apply(lambda x: round(x, 2))
        cumplimiento_recaudo_por_division['Contexto'] = cumplimiento_recaudo_por_division['Division']
        cumplimiento_recaudo_por_division['Variable'] = 'VentaCobradaDivision'
        
        cumplimiento_recaudo_por_division = cumplimiento_recaudo_por_division[['Contexto', 'Variable', 'Fecha', 'Real', 'Presupuesto', 'PorcentajeCumplimiento']]
        
        # Checkpoint Calculo
        return self.generacion_columnas(cumplimiento_recaudo_por_division)
    
    def calcular1(self, grupo_empleados = None, *args, **kwargs):
        
        df_venta_recaudo_real = self.dataframe_loader.df_venta_recaudo_real
        df_venta_recaudo_presupuesto = self.dataframe_loader.df_venta_recaudo_presupuesto
        
        recaudo_real_por_division = df_venta_recaudo_real[['Division', 'Fecha', 'Recaudo']]
        recaudo_presupuesto_por_division = df_venta_recaudo_presupuesto[['Division', 'Fecha', 'Recaudo']]
        
        
        recaudo_real_por_division = recaudo_real_por_division.groupby(['Division', 'Fecha'], as_index=False).agg({'Recaudo': 'sum'})
        recaudo_real_por_division['Real'] = recaudo_real_por_division['Recaudo']
        recaudo_real_por_division.drop(columns=['Recaudo'], inplace=True)

        recaudo_presupuesto_por_division = recaudo_presupuesto_por_division.groupby(['Division', 'Fecha'], as_index=False).agg({'Recaudo': 'sum'})
        recaudo_presupuesto_por_division['Presupuesto'] = recaudo_presupuesto_por_division['Recaudo']
        recaudo_presupuesto_por_division.drop(columns=['Recaudo'], inplace=True)

        cumplimiento_recaudo_por_division = pd.merge(
            left=recaudo_presupuesto_por_division,
            right=recaudo_real_por_division,
            on=['Division', 'Fecha'],
            how='left'
        )
        cumplimiento_recaudo_por_division['Real'].fillna(0.0, inplace=True)
        cumplimiento_recaudo_por_division['Real'] = cumplimiento_recaudo_por_division.groupby(['Division'])['Real'].transform(pd.Series.cumsum)
        cumplimiento_recaudo_por_division['Presupuesto'] = cumplimiento_recaudo_por_division.groupby(['Division'])['Presupuesto'].transform(pd.Series.cumsum)
        cumplimiento_recaudo_por_division['PorcentajeCumplimiento'] = cumplimiento_recaudo_por_division['Real'] / cumplimiento_recaudo_por_division['Presupuesto']
        cumplimiento_recaudo_por_division['PorcentajeCumplimiento'] = cumplimiento_recaudo_por_division['PorcentajeCumplimiento'].apply(lambda x: round(x, 2))

        cumplimiento_recaudo_por_division['Contexto'] = cumplimiento_recaudo_por_division['Division']
        cumplimiento_recaudo_por_division['Variable'] = 'VentaCobradaDivision'
        cumplimiento_recaudo_por_division = cumplimiento_recaudo_por_division[['Contexto', 'Variable', 'Fecha', 'Real', 'Presupuesto', 'PorcentajeCumplimiento']]
        return cumplimiento_recaudo_por_division

In [170]:
venta_cobrada_division = VentaCobradaDivision(dataframe_loader)
venta_cobrada_division.calcular(None, c_area_calculo_sba).head()

Series([], Name: TipoEmpleado, dtype: object)


,Contexto,Variable,Fecha,Real,Presupuesto,PorcentajeCumplimiento
0,AE,VentaCobradaDivision,2023-01-01,3.372108e+09,2.772690e+09,1.22
1,AE,VentaCobradaDivision,2023-02-01,6.398180e+09,6.466029e+09,0.99
2,AE,VentaCobradaDivision,2023-03-01,8.915028e+09,9.793257e+09,0.91
3,AE,VentaCobradaDivision,2023-04-01,1.220146e+10,1.312049e+10,0.93
4,AE,VentaCobradaDivision,2023-05-01,1.767949e+10,1.607802e+10,1.10


In [171]:
venta_cobrada_division.calcular1(None, c_area_calculo_sba).head()

,Contexto,Variable,Fecha,Real,Presupuesto,PorcentajeCumplimiento
0,AE,VentaCobradaDivision,2023-01-01,3.372108e+09,2.772690e+09,1.22
1,AE,VentaCobradaDivision,2023-02-01,6.398180e+09,6.466029e+09,0.99
2,AE,VentaCobradaDivision,2023-03-01,8.915028e+09,9.793257e+09,0.91
3,AE,VentaCobradaDivision,2023-04-01,1.220146e+10,1.312049e+10,0.93
4,AE,VentaCobradaDivision,2023-05-01,1.767949e+10,1.607802e+10,1.10


In [ ]:
# venta_facturada_unidad_negocio_sub = VentaFacturadaUnidadNegocioSub(dataframe_loader)
# venta_facturada_unidad_negocio_sub.calcular(None, c_area_calculo_sba)

In [ ]:
# df_parrillas = dataframe_loader.df_parrillas

# # Ahora, crea un diccionario que mapea cada 'rubro' a la clase correspondiente.
# estrategias_calculo = {
#     'VentaFacturadaZonaPais': VentaFacturadaZonaPais(dataframe_loader),
#     'VentaFacturadaPortafolioCluster': VentaFacturadaPortafolioCluster(dataframe_loader),
#     # ... y así sucesivamente para cada 'rubro'
# }

# # Función para aplicar el cálculo utilizando las clases
# def aplicar_calculo(rubro, grupo_datos):
#     estrategia_calculo = estrategias_calculo.get(rubro, CalculoRubroBase())
#     return estrategia_calculo.calcular(grupo_datos)

# # Ciclo para aplicar el cálculo para cada rubro
# for rubro in df_parrillas['Variable'].drop_duplicates():
#     grupo_datos = df_parrillas[df_parrillas['Variable'] == rubro]
#     resultado_calculo = aplicar_calculo(rubro, grupo_datos)
#     print(f"{rubro}: {resultado_calculo}")